In [ ]:
from iobjectspy import (Point2D,
                        QueryParameter,
                        open_datasource,
                        create_datasource,
                        SpatialQueryMode)
import os

# 设置示例数据路径
example_data_dir = ''

# 设置结果输出路径
out_dir = os.path.join(example_data_dir, 'out')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)


def _write_result_recordset(recordset):
    """将查询的结果记录集写入数据源中"""

    record_out_path = os.path.join(out_dir, 'out_query_data.udb')
    if not os.path.exists(record_out_path):
        ds = create_datasource(record_out_path)
    else:
        ds = open_datasource(record_out_path)
    assert ds is not None, '打开结果数据源失败'

    result_dt = ds.write_recordset(recordset, out_dataset_name=recordset.dataset.name + '_query')

    if result_dt is not None:
        print('写入查询结果记录集到数据集 ' + ds.connection_info.server + '|' + result_dt.name + '　成功')
    else:
        print('写入查询结果记录集失败')
    ds.close()


def attribute_filter_query_test():
    """对数据集 landr 进行属性查询,并将查询结果输出到结果数据源中"""

    ds = open_datasource(os.path.join(example_data_dir, 'example_data.udb'))
    assert ds is not None, '打开数据源失败'

    dataset = ds['landr']
    assert dataset is not None, '获取数据集失败'

    recordset = dataset.query_with_filter('R_AREA > 500 and R_AREA < 5000', cursor_type='STATIC')
    assert recordset is not None, '查询记录集失败'

    print('属性查询结果记录数为: ' + str(recordset.get_record_count()))

    _write_result_recordset(recordset)

    ds.close()


def bounds_query_test():
    """对数据集 landr 进行地理范围查询,并将查询结果输出到结果数据源中"""

    ds = open_datasource(os.path.join(example_data_dir, 'example_data.udb'))
    assert ds is not None, '打开数据源失败'

    dataset = ds['landr']
    assert dataset is not None, '获取数据集失败'

    rc = ds['adminR'].get_geometries('SmID == 2')[0].bounds
    recordset = dataset.query_with_bounds(rc, cursor_type='STATIC')
    assert recordset is not None, '查询记录集失败'

    print('范围查询结果记录数为: ' + str(recordset.get_record_count()))

    _write_result_recordset(recordset)

    ds.close()


def distance_query_test():
    """对数据集 landp 进行距离查询, 查询距离为100米, 并将查询结果输出到结果数据源中"""

    ds = open_datasource(os.path.join(example_data_dir, 'example_data.udb'))
    assert ds is not None, '打开数据源失败'

    dataset = ds['landp']
    assert dataset is not None, '获取数据集失败'

    recordset = dataset.query_with_distance(Point2D(315.782892179537, 260.119529494306), 100, unit='Meter',
                                            cursor_type='STATIC')
    assert recordset is not None, '查询记录集失败'

    print('距离查询结果记录数为: ' + str(recordset.get_record_count()))

    _write_result_recordset(recordset)

    ds.close()


def spatial_query_test():
    """对数据集 landp 进行空间查询查询, 并将查询结果输出到结果数据源中"""

    ds = open_datasource(os.path.join(example_data_dir, 'example_data.udb'))
    assert ds is not None, '打开数据源失败'

    dataset = ds['landp']
    assert dataset is not None, '获取数据集失败'

    query_geo = ds['adminR'].get_geometries('SmID == 2')[0]
    query_parameter = (QueryParameter().set_spatial_query_mode(SpatialQueryMode.CONTAIN).
                       set_spatial_query_object(query_geo).set_cursor_type('static'))

    recordset = dataset.query(query_parameter)

    assert recordset is not None, '查询记录集失败'

    print('空间查询结果记录数为: ' + str(recordset.get_record_count()))

    _write_result_recordset(recordset)

    ds.close()


if __name__ == '__main__':
    # 属性查询
    attribute_filter_query_test()

    # 范围查询
    bounds_query_test()

    # 距离查询
    distance_query_test()

    # 空间查询
    spatial_query_test()


In [ ]:
exit()